In [5]:
import spacy
import random
from spacy.util import minibatch, compounding

In [ ]:
nlp = spacy.load("pt_core_news_sm")
doc = nlp(text)
 
print(nlp.pipe_names)

In [14]:
def train_model(data_dict, iterations):
    # Cria um modelo vazio do Spacy
    nlp = spacy.blank("pt")
    
    # Cria um ner (Named Entity Recognizer) vazio e adiciona ao pipeline do modelo
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)

    # Define as labels (entidades) para treinar o modelo
    for label in data_dict.keys():
        ner.add_label(label)
    
    # Prepara os dados de treinamento
    train_data = []
    for label, examples in data_dict.items():
        for text, annotations in examples:
            train_data.append((text, annotations))
    
    # Inicia o treinamento do modelo
    nlp.begin_training()
    for itn in range(iterations):
        # Embaralha os dados de treinamento para aumentar a diversidade
        random.shuffle(train_data)
        losses = {}
        
        # Divide os dados em lotes (batches) e atualiza o modelo em cada lote
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, losses=losses)
        
        # Imprime a perda (loss) durante o treinamento
        print("Iteration:", itn+1, "Loss:", losses)

    # # Salvando o modelo treinado
    # nlp.to_disk(output_dir)

    return nlp


#### Exemplo de dados dos parâmetros:

In [15]:
data_dict = entidades = {
    "BOMBA": [
    ("Bomba centrífuga", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de água", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de vácuo", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de calor", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba dosadora", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba submersível", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de pistão", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de diafragma", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de engrenagem", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de incêndio", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba centrífuga", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de água", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de vácuo", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de calor", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba dosadora", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba submersível", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de pistão", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de diafragma", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de engrenagem", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de incêndio", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de vácuo rotativa", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de alta pressão", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de transferência", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de ar comprimido", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de esgoto", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba peristáltica", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de irrigação", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de deslocamento positivo", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de lóbulos", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de turbina", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de palhetas", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de cavidade progressiva", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba hidráulica", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de duplo diafragma", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de poço", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de drenagem", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de concreto", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de óleo", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de químicos", {"entities": [(0, 5, "BOMBA")]}),
    ("Bomba de refrigeração", {"entities": [(0, 5, "BOMBA")]}),]
,
"ROLAMENTO": [
    ("Rolamento de esferas", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de rolos", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento axial", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento radial", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de agulhas", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento autocompensador", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de contato angular", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de precisão", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento magnético", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento cerâmico", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento blindado", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento vedado", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento linear", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de alta temperatura", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de baixo atrito", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de longa vida útil", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de elevada capacidade de carga", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de alta velocidade", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de ROLAMENTO axial de rolos cilíndricos", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de rolo cilíndrico", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de esferas de contato angular de quatro pontos", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de rolo esférico", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de giro", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento para máquinas pesadas", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento para alta precisão", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento para altas temperaturas", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento para baixo ruído", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento para aplicações industriais", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de transmissão de potência", {"entities": [(0, 9, "ROLAMENTO")]}),
    ("Rolamento de movimentação linear", {"entities": [(0, 9, "ROLAMENTO")]}),]
,
"VALVULA": [
    ("Válvula de controle", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de alívio", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula solenóide", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de esfera", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de borboleta", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de retenção", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de gaveta", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de diafragma", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de agulha", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de segurança", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de escape", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de bloqueio", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula reguladora de pressão", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de descarga", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de enchimento", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de descarga a vácuo", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de reabastecimento", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de admissão", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de exaustão", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de pressão diferencial", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de controle de vazão", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de alívio de pressão", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de retenção de pé", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de alívio térmico", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de retenção de disco duplo", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de controle direcional", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de retenção de pistão", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de segurança de pressão", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de controle proporcional", {"entities": [(0, 7, "VALVULA")]}),
    ("Válvula de isolamento", {"entities": [(0, 7, "VALVULA")]}),
],
    "ACCOR" : [("Acionamento por corrente", {"entities": [(0, 23, "ACCOR")]}),
    ("Motor de acionamento por corrente", {"entities": [(12, 35, "ACCOR")]}),
    ("Acionamento por corrente elétrica", {"entities": [(0, 23, "ACCOR")]}),
    ("Corrente de acionamento", {"entities": [(0, 22, "ACCOR")]}),
    ("Sistema de acionamento por corrente", {"entities": [(21, 44, "ACCOR")]}),
    ("Acionamento por corrente contínua", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente alternada", {"entities": [(0, 23, "ACCOR")]}),
    ("Corrente de acionamento elétrico", {"entities": [(0, 27, "ACCOR")]}),
    ("Acionamento por corrente trifásica", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente monofásica", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente direta", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente reversa", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente pulsada", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente de alta tensão", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente de baixa tensão", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente constante", {"entities": [(0, 23, "ACCOR")]}),
    ("Corrente de acionamento ajustável", {"entities": [(0, 27, "ACCOR")]}),
    ("Acionamento por corrente variável", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente programável", {"entities": [(0, 23, "ACCOR")]}),
    ("Corrente de acionamento estável", {"entities": [(0, 25, "ACCOR")]}),
    ("Acionamento por corrente de precisão", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente de alta potência", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente de baixa potência", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente de média potência", {"entities": [(0, 23, "ACCOR")]}),
    ("Corrente de acionamento industrial", {"entities": [(0, 27, "ACCOR")]}),
    ("Acionamento por corrente de tração", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente de transporte", {"entities": [(0, 23, "ACCOR")]}),
    ("Acionamento por corrente de elevação", {"entities": [(0, 23, "ACCOR")]}),
    ("Corrente de acionamento automático", {"entities": [(0, 27, "ACCOR")]}),
    ("Acionamento por corrente de controle", {"entities": [(0, 23, "ACCOR")]})
    ],
    "CAIXA": [
        ("Caixa de engrenagens", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens industriais", {"entities": [(0, 11, "CAIXA")]}),
        ("Sistema de transmissão por engrenagens", {"entities": [(24, 35, "CAIXA")]}),
        ("Caixa de engrenagens helicoidais", {"entities": [(0, 14, "CAIXA")]}),
        ("Caixa de engrenagens cônicas", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de alta precisão", {"entities": [(0, 11, "CAIXA")]}),
        ("Caixa de engrenagens de redução", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de grande torque", {"entities": [(0, 11, "CAIXA")]}),
        ("Caixa de engrenagens de alta potência", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de baixo ruído", {"entities": [(0, 11, "CAIXA")]}),
        ("Caixa de engrenagens de velocidade variável", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de transmissão", {"entities": [(0, 11, "CAIXA")]}),
        ("Caixa de engrenagens de eixos paralelos", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de dentes retos", {"entities": [(0, 11, "CAIXA")]}),
        ("Caixa de engrenagens planetárias", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de eixos cruzados", {"entities": [(0, 11, "CAIXA")]}),
        ("Caixa de engrenagens de precisão", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de múltiplas velocidades", {"entities": [(0, 11, "CAIXA")]}),
        ("Caixa de engrenagens de montagem fácil", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de longa vida útil", {"entities": [(0, 11, "CAIXA")]}),
        ("Caixa de engrenagens de operação suave", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de alto desempenho", {"entities": [(0, 11, "CAIXA")]}),
        ("Caixa de engrenagens de serviço pesado", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de reversão", {"entities": [(0, 11, "CAIXA")]}),
        ("Caixa de engrenagens de transmissão de potência", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de lubrificação permanente", {"entities": [(0, 11, "CAIXA")]}),
        ("Caixa de engrenagens de carga elevada", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de troca rápida", {"entities": [(0, 11, "CAIXA")]}),
        ("Caixa de engrenagens de operação silenciosa", {"entities": [(0, 14, "CAIXA")]}),
        ("Engrenagens de baixa manutenção", {"entities": [(0, 11, "CAIXA")]})
    ]
}

output_dir = r"..\documents"

iterations = 100
model = train_model(data_dict, iterations)

ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <spacy.pipeline.ner.EntityRecognizer object at 0x000001DDB247B350> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.